## Risk Generation Prompt Engineering Notes

Since the risk generation is something we are adding to our project to help visualize the current risks an organization has, there is no initial data, examples, or templates. We will have to research and generate our own examples.